### Carbon Python Sample No.3 - SPSS to Heat Map Chart

TODO: Add an introduction

:star: The 21MB binary data file of SPSS data used by the example can be downloaded from: [Demo_21_24_with_caseid.sav](https://systemrcs.blob.core.windows.net/download/Demo_21_24_with_Caseid.sav)

In [1]:
import pandas as pd
import os
import numpy as np
import itertools
import plotly.express as px
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import re
import pyreadstat
from pandas.api.types import CategoricalDtype

from natsort import natsorted, index_natsorted, order_by_index

import requests
from collections import Counter #(capital 'C')
NoAnsString='Unknown'


import PyCommon as pc

def naturalSort(dfdf:pd.DataFrame):
    dfdf = dfdf.reindex(index=natsorted(dfdf.index))
    return dfdf

### Read the data in from an SPSS.sav file

In [2]:
df,meta= pyreadstat.read_sav(r'Demo_21_24_with_caseid.sav',apply_value_formats=True, 
                                formats_as_category=True, formats_as_ordered_category=False)
df[:2]

,DayNum,IND,PRMSHP,CCF,CPF,WUBY,WUBYsup,TMBA,UOBA_1,UOBA_2,...,GEN,Married,Household,HHSZ,NCHLD,EDU,INC,Income,RawIncome,Indent
0,04-Jan-21,None,Yes,At least once a day,A few times a month,Department Store,,Brand7,Brand2,Brand4,...,Female,No,Middle Family,3,4,HIGH SCHOOL,$30k to 50k,$4000,4000.0,1.0
1,04-Jan-21,None,Yes,Several times a week,A few times a month,Convenience Store,,Brand7,Brand2,Brand4,...,Male,Yes,Middle Family,3,2,BACHELOR,Over $75k,$9000,9000.0,2.0


In [3]:
meta.column_names

['DayNum',
 'IND',
 'PRMSHP',
 'CCF',
 'CPF',
 'WUBY',
 'WUBYsup',
 'TMBA',
 'UOBA_1',
 'UOBA_2',
 'UOBA_3',
 'UOBA_4',
 'UOBA_5',
 'UOBA_6',
 'TMAA',
 'UOAA_1',
 'UOAA_2',
 'UOAA_3',
 'ABA_1',
 'ABA_2',
 'ABA_3',
 'AAATV_1',
 'AAATV_2',
 'AAATV_3',
 'AAAOM_1',
 'AAAOM_2',
 'AAAOM_3',
 'BIM_1_1',
 'BIM_1_2',
 'BIM_1_3',
 'BIM_1_4',
 'BIM_1_5',
 'BIM_1_6',
 'BIM_1_7',
 'BIM_1_8',
 'BIM_1_9',
 'BIM_1_10',
 'BIM_2_1',
 'BIM_2_2',
 'BIM_2_3',
 'BIM_2_4',
 'BIM_2_5',
 'BIM_2_6',
 'BIM_2_7',
 'BIM_2_8',
 'BIM_2_9',
 'BIM_2_10',
 'BIM_3_1',
 'BIM_3_2',
 'BIM_3_3',
 'BIM_3_4',
 'BIM_3_5',
 'BIM_3_6',
 'BIM_3_7',
 'BIM_3_8',
 'BIM_3_9',
 'BIM_3_10',
 'BEQ_1_1',
 'BEQ_1_2',
 'BEQ_1_3',
 'BEQ_1_4',
 'BEQ_1_5',
 'BEQ_1_6',
 'BEQ_1_7',
 'BEQ_2_1',
 'BEQ_2_2',
 'BEQ_2_3',
 'BEQ_2_4',
 'BEQ_2_5',
 'BEQ_2_6',
 'BEQ_2_7',
 'BEQ_3_1',
 'BEQ_3_2',
 'BEQ_3_3',
 'BEQ_3_4',
 'BEQ_3_5',
 'BEQ_3_6',
 'BEQ_3_7',
 'BratRats_1_1',
 'BratRats_1_2',
 'BratRats_2_1',
 'BratRats_2_2',
 'NPS_1',
 'NPS_2',
 'NPS_3',
 

### Change NaNs to the NoAnsString

In [4]:
def isNaN(x):
    return x!=x

def removeCatNaNs(nas:str):
    global df
    for col in df.columns:
        entries=[]
        if df[col].isna().values.any():
            for r in df[col]:
                if isNaN(r):
                    entries.append(nas)
                else:
                    entries.append(r)
        if len(entries) == len(df):
            df[col]=pd.Categorical(entries)
    return df
    
df=removeCatNaNs(NoAnsString)
df[:2]

,DayNum,IND,PRMSHP,CCF,CPF,WUBY,WUBYsup,TMBA,UOBA_1,UOBA_2,...,GEN,Married,Household,HHSZ,NCHLD,EDU,INC,Income,RawIncome,Indent
0,04-Jan-21,None,Yes,At least once a day,A few times a month,Department Store,,Brand7,Brand2,Brand4,...,Female,No,Middle Family,3,4,HIGH SCHOOL,$30k to 50k,$4000,4000.0,1.0
1,04-Jan-21,None,Yes,Several times a week,A few times a month,Convenience Store,,Brand7,Brand2,Brand4,...,Male,Yes,Middle Family,3,2,BACHELOR,Over $75k,$9000,9000.0,2.0


### The API call is wrapped into the function carboncrosstab

In [5]:
table = pc.carboncrosstab(df.GEN,df['Age'],True)
table=naturalSort(table)
table

,Female,Male
15-25,19.46,19.14
26-35,19.64,18.68
36-50,20.28,19.92
51-65,19.80,20.88
65+,20.82,21.38


In [6]:
monthdict={1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}
# date=[]
monthyear=[]
for d in df['DayNum']:
    dateparts=d.split('-')
    monthyear.append(str(dateparts[1] + '-' + dateparts[2]))
df['monthyear']=monthyear


### Multi-coded variables are very common in Market Research
  
  In this example we are combinning 'Top of Mind' Awareness ('TMBA'), with 'Other Bransds aware of' (UOBA_1 to UOBA_6).
    
All the variales have the same code frame, as defined in 'BrandDict'

In [7]:
mvars=['UOBA','UOAA','ABA','AAATV','AAAOM']


for mvar in mvars:
    targetcols=[]
    var=[]
    for coll in df.columns:
        if coll.startswith(mvar + '_'):
            targetcols.append(coll)
    tc=df[targetcols].T
    for resp in tc.columns:
        respentry=tc[resp].to_list()
        while NoAnsString in respentry:
            respentry.remove(NoAnsString)
        var.append(respentry)
    df[mvar]=var


### Combine TMBA and the new UOBA to create UBA

TMBA = Top of mind awareness, the first brand mentioned

UOBA = Other Brands mentioned

UBA = UnAided Brand Awareness

In [8]:
var=[]
tc=df[['TMBA','UOBA']].T
for resp in tc.columns:
    respentry=tc[resp][1]
    respentry.append(tc[resp][0])
    ree=np.array(respentry)
    r=np.unique(ree).tolist()
    var.append(r[1:])

df['UBA']=var

df['UBA']

0               [Brand2, Brand4, Brand7]
1               [Brand2, Brand4, Brand7]
2       [Brand2, Brand6, Brand7, Brand8]
3                      [Brand10, Brand2]
4       [Brand1, Brand2, Brand4, Brand6]
                      ...               
9995    [Brand1, Brand3, Brand5, Brand6]
9996    [Brand2, Brand4, Brand5, Brand7]
9997    [Brand1, Brand2, Brand3, Brand4]
9998            [Brand1, Brand2, Brand5]
9999    [Brand1, Brand2, Brand3, Brand8]
Name: UBA, Length: 10000, dtype: object

In [9]:
tab1=pc.carboncrosstab(df['GEN'], df['UBA'], True)
tab1=naturalSort(tab1)
tab1

,Female,Male
Brand1,67.04,66.10
Brand2,46.44,47.10
Brand3,58.40,60.02
Brand4,50.43,51.82
Brand5,37.07,37.93
Brand6,28.00,28.02
Brand7,19.50,18.96
Brand8,11.72,11.45
Brand9,7.76,6.72
Brand10,12.56,13.48


### An NPS calculation

In [10]:
subBrandDict = {1:'Brand1', 2 : 'Brand2', 3:'Brand3'}
for b in subBrandDict:
    nps=[]
    for i,r in df.iterrows():
        entry=[]
        score=r['NPS_' + str(b)]
        entry.append(score)
        npstype='Passive'
        if int(score) <=6:
            npstype='Detractor'
        if int(score) >=9:
            npstype='Promoter'
        entry.append(npstype)
        nps.append(entry)
    df['nps_' + subBrandDict.get(b)]=nps
    
df['nps_Brand1']

0       [1, Detractor]
1       [2, Detractor]
2       [1, Detractor]
3       [2, Detractor]
4         [8, Passive]
             ...      
9995    [2, Detractor]
9996    [10, Promoter]
9997    [2, Detractor]
9998    [3, Detractor]
9999    [4, Detractor]
Name: nps_Brand1, Length: 10000, dtype: object

In [11]:
table=naturalSort(pc.carboncrosstab(df['GEN'],df['nps_Brand1'], True))
table

,Female,Male
1,2.93,2.83
2,11.51,11.41
3,4.21,4.29
4,5.40,4.97
5,6.40,6.56
6,10.05,9.43
7,7.66,8.39
8,19.52,19.72
9,23.37,23.43
10,8.95,8.97


In [12]:
def makeBanner():
    bannercol=[]
    for i,r in df.iterrows():
        respentry=[]
        respentry.append('Total')

        if r['GEN']=='Males':
            respentry.append('Male')
        else:
            respentry.append('Female')

        respentry.append(r['Age'])
        respentry.append(r['Region'])
        # respentry.append(r['BBL'])

        bannercol.append(respentry)
    return bannercol

df['banner']=makeBanner()
df['banner']

0       [Total, Female, 36-50, SE]
1       [Total, Female, 51-65, SE]
2       [Total, Female, 26-35, NW]
3       [Total, Female, 15-25, NW]
4       [Total, Female, 26-35, SE]
                   ...            
9995    [Total, Female, 15-25, NW]
9996      [Total, Female, 65+, SW]
9997    [Total, Female, 51-65, NW]
9998      [Total, Female, 65+, NE]
9999    [Total, Female, 15-25, NW]
Name: banner, Length: 10000, dtype: object

In [13]:

ct= pc.carboncrosstab(df['monthyear'],df['UBA'], True)
ct=naturalSort(ct).round(1)

cm = sns.light_palette("blue", as_cmap=True)
ct.style.background_gradient(cmap=cm,subset=(ct.index[:], ct.select_dtypes(float).columns)).applymap(lambda v: 'opacity: 50%;' if (v < 10) else None ).format(precision=1)


,Jan-21,Feb-21,Mar-21,Apr-21,May-21,Jun-21,Jul-21,Aug-21,Sep-21,Oct-21,Nov-21,Dec-21,Jan-22,Feb-22,Mar-22,Apr-22,May-22,Jun-22,Jul-22,Aug-22,Sep-22,Oct-22,Nov-22,Dec-22,Jan-23,Feb-23,Mar-23,Apr-23,May-23,Jun-23,Jul-23,Aug-23,Sep-23,Oct-23,Nov-23,Dec-23,Jan-24,Feb-24,Mar-24,Apr-24,May-24,Jun-24,Jul-24,Aug-24,Sep-24,Oct-24,Nov-24
Brand1,68.0,65.0,66.1,60.4,60.0,66.8,60.4,65.0,61.4,66.2,62.7,60.0,66.2,65.5,65.6,64.3,62.3,69.6,64.8,64.8,60.9,72.9,68.6,71.8,63.6,70.5,66.1,68.0,71.7,69.6,70.0,73.9,65.7,67.3,72.3,67.5,70.9,69.5,72.4,72.7,70.4,74.0,63.0,64.1,63.3,60.0,63.1
Brand2,44.0,47.0,43.0,50.4,47.6,47.7,50.9,44.6,44.6,46.2,46.8,48.3,43.8,47.5,48.3,46.7,43.6,47.3,48.1,46.1,47.3,51.9,40.4,51.2,45.9,45.0,45.6,44.0,47.0,48.6,46.2,44.4,44.8,49.1,46.4,49.4,47.0,46.7,49.0,46.4,48.3,43.5,43.0,49.1,52.9,51.3,41.9
Brand3,57.0,59.0,56.1,59.1,62.4,52.7,54.6,59.1,62.3,56.2,55.4,59.4,61.4,58.0,49.6,61.0,67.3,55.9,59.0,58.3,63.2,59.5,53.2,58.2,60.4,56.0,61.7,61.0,57.8,59.6,56.7,60.0,63.3,59.1,62.3,66.2,58.7,57.6,54.3,65.9,59.6,59.5,63.0,63.6,56.2,60.0,65.6
Brand4,49.0,47.5,55.2,50.9,55.7,53.2,51.4,55.0,56.8,55.7,52.7,55.6,50.0,51.0,56.1,55.2,50.0,47.7,49.0,51.7,47.7,56.2,54.1,50.0,49.1,45.0,50.0,50.5,47.4,45.0,53.8,43.9,50.0,56.8,47.3,42.5,49.6,51.4,52.9,43.2,48.3,51.0,53.5,52.3,56.2,49.6,55.6
Brand5,35.0,38.0,34.8,36.4,36.7,40.9,45.9,37.7,35.0,37.1,42.7,42.8,32.9,34.5,41.3,41.0,37.3,40.4,40.5,36.1,36.8,29.5,41.4,37.6,40.9,37.0,36.5,39.0,31.3,35.9,35.7,37.0,30.5,35.0,42.3,41.2,40.9,37.1,38.6,38.6,37.0,39.5,31.7,35.9,37.1,37.4,35.0
Brand6,25.5,27.0,27.8,26.8,26.7,26.8,28.2,26.4,26.8,27.1,32.3,25.6,31.0,25.0,30.0,27.6,33.6,30.0,31.9,29.6,25.4,24.8,23.2,22.4,30.0,29.5,30.4,27.5,24.8,29.6,28.1,33.9,27.6,26.4,27.7,25.0,27.8,29.5,31.4,24.6,36.5,29.5,25.2,26.4,27.1,25.6,27.5
Brand7,18.0,17.5,23.0,24.1,18.1,21.4,19.6,23.2,16.8,17.6,24.6,20.0,23.3,19.5,18.3,20.5,17.3,21.8,19.5,15.6,16.4,19.5,16.4,18.2,19.6,19.0,16.5,19.0,16.5,15.9,19.5,17.4,18.6,19.1,21.4,18.1,19.1,16.2,13.3,18.2,19.6,17.5,20.0,17.7,26.2,23.0,22.5
Brand8,9.5,10.5,10.4,15.0,10.0,12.7,13.6,12.7,12.3,14.8,7.7,10.6,12.9,12.0,10.0,13.8,11.8,14.1,12.9,10.4,11.4,12.9,14.1,10.6,14.1,10.5,10.0,9.0,11.7,13.2,15.7,12.2,10.0,10.4,9.1,6.9,8.7,11.4,8.6,8.6,13.5,10.0,13.9,12.3,11.9,11.7,13.1
Brand9,8.5,9.5,5.6,7.7,8.6,5.4,5.9,7.7,8.2,5.7,10.0,6.7,6.2,8.5,7.8,6.7,8.2,5.0,4.8,6.1,10.4,3.8,9.6,7.1,5.9,8.5,7.0,9.5,8.7,7.7,7.1,7.0,6.2,5.0,8.6,7.5,9.6,6.2,9.0,8.2,5.6,8.5,6.1,7.7,5.7,6.1,5.0
Brand10,14.0,10.5,13.0,14.6,14.8,11.4,15.4,10.9,12.7,13.3,10.4,12.8,16.7,14.0,13.5,10.5,9.6,11.8,13.8,11.7,18.2,14.3,14.1,12.4,13.2,9.5,13.5,13.5,13.0,10.9,14.3,13.9,17.1,12.7,10.9,14.4,10.0,10.5,13.8,12.3,9.1,13.0,15.2,16.4,13.3,15.2,11.9


In [14]:
import plotly.express as px

fig=px.line(ct.T)  # Need to transpose the crosstab
fig.show()

